# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,albany,42.6001,-73.9662,18.58,78,1,1.60,US,1723084218
1,1,notodden,59.5594,9.2585,12.78,84,99,1.94,NO,1723084219
2,2,yellowknife,62.4560,-114.3525,21.90,46,20,5.14,CA,1723084220
3,3,kapuskasing,49.4169,-82.4331,18.05,72,100,1.54,CA,1723083761
4,4,kibiti,-7.7218,38.9375,18.32,97,95,1.25,TZ,1723084222


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 520,
    size = "Humidity",
    scale = 1,
    color = "City"
    
      
)

# Display the map
city_data_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city = city_data_df.loc[(city_data_df["Max Temp"] < 32) & (city_data_df["Max Temp"] > 21) & (city_data_df["Cloudiness"] < 70) & (city_data_df["Wind Speed"] < 10) & (city_data_df["Humidity"] < 50)]

# Drop any rows with null values
ideal_city = ideal_city.dropna()

# Display sample data
ideal_city.head(20)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,yellowknife,62.4560,-114.3525,21.90,46,20,5.14,CA,1723084220
48,48,al jawf,29.5000,38.7500,23.87,40,1,3.20,SA,1723084271
129,129,imperatriz,-5.5264,-47.4917,23.18,42,3,1.21,BR,1723084368
148,148,baharly,38.4362,57.4316,31.32,42,52,1.21,TM,1723084389
158,158,hotan,37.0997,79.9269,29.29,18,32,2.44,CN,1723084400
168,168,moranbah,-22.0016,148.0466,24.52,43,13,6.53,AU,1723084411
169,169,whitehorse,60.7161,-135.0538,27.79,30,0,5.66,CA,1723084412
174,174,alice springs,-23.7000,133.8833,23.75,27,0,1.54,AU,1723084418
182,182,fort st. john,56.2499,-120.8529,22.94,43,20,4.12,CA,1723084394
184,184,novyy urgal,51.0736,132.5642,28.87,42,8,0.50,RU,1723084429


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = None

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
2,yellowknife,CA,62.4560,-114.3525,46,None
48,al jawf,SA,29.5000,38.7500,40,None
129,imperatriz,BR,-5.5264,-47.4917,42,None
148,baharly,TM,38.4362,57.4316,42,None
158,hotan,CN,37.0997,79.9269,18,None
168,moranbah,AU,-22.0016,148.0466,43,None
169,whitehorse,CA,60.7161,-135.0538,30,None
174,alice springs,AU,-23.7000,133.8833,27,None
182,fort st. john,CA,56.2499,-120.8529,43,None
184,novyy urgal,RU,51.0736,132.5642,42,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
yellowknife - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
imperatriz - nearest hotel: La Bella Hotel
baharly - nearest hotel: No hotel found
hotan - nearest hotel: Hua Yi International Hotel (accepts foreigners)
moranbah - nearest hotel: No hotel found
whitehorse - nearest hotel: Town & Mountain Hotel
alice springs - nearest hotel: Aurora Alice Springs
fort st. john - nearest hotel: Fort St. John Motor Inn
novyy urgal - nearest hotel: No hotel found
mount isa - nearest hotel: Ibis Styles
raton - nearest hotel: Travelodge
bodaybo - nearest hotel: НОЙ
idri - nearest hotel: No hotel found
zouerate - nearest hotel: فندق تيرس زمور
coari - nearest hotel: Hotel Saúde
biloela - nearest hotel: Settlers Hotel Motel
vernon - nearest hotel: Okanagan Royal Park Inn
susanville - nearest hotel: SureStay Plus Hotel by Best Western Susanville
vilhena - nearest hotel: No hotel found
sao felix do xingu - nearest hotel: Hotel Terraço
karratha - nearest hotel

,City,Country,Lat,Lng,Humidity,Hotel Name
2,yellowknife,CA,62.4560,-114.3525,46,No hotel found
48,al jawf,SA,29.5000,38.7500,40,No hotel found
129,imperatriz,BR,-5.5264,-47.4917,42,La Bella Hotel
148,baharly,TM,38.4362,57.4316,42,No hotel found
158,hotan,CN,37.0997,79.9269,18,Hua Yi International Hotel (accepts foreigners)
168,moranbah,AU,-22.0016,148.0466,43,No hotel found
169,whitehorse,CA,60.7161,-135.0538,30,Town & Mountain Hotel
174,alice springs,AU,-23.7000,133.8833,27,Aurora Alice Springs
182,fort st. john,CA,56.2499,-120.8529,43,Fort St. John Motor Inn
184,novyy urgal,RU,51.0736,132.5642,42,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_df_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 520,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols=["Lat","Lng","City","Humidity","Hotel Name","Country"] 
)

# Display the map
hotel_df_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)